In [1]:
import os, time, json, csv
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service  # Selenium Manager

# --- CONFIG ---
BASE = "https://sistema.buenavistacapital.com"
LIST_TEMPLATE = BASE + "/admin/informacion_financiera/activo/?p={page}"
OUTFILE_JSON = "activos_from_profile.json"
OUTFILE_CSV  = "activos_from_profile.csv"
OUTFILE_DICT = "activos_dict.json"
OUTFILE_XLSX = "activos2.xlsx"

# 54 páginas totales (0-index): 0..53
TOTAL_PAGES = 54

FIREFOX_PROFILE_PATH = "/Users/matias/Library/Application Support/Firefox/Profiles/a6k7okua.default-release-1"
assert os.path.isdir(FIREFOX_PROFILE_PATH), "Perfil no existe"

opts = Options()
# opts.headless = True  # primero prueba sin headless para confirmar sesión
opts.profile = FIREFOX_PROFILE_PATH
# Ayuda con errores de sockets en macOS
opts.set_preference("network.http.max-persistent-connections-per-server", 2)
opts.set_preference("network.http.max-persistent-connections-per-proxy", 2)

# Selenium 4.6+: Selenium Manager resuelve geckodriver
service = Service()
driver  = webdriver.Firefox(service=service, options=opts, keep_alive=False)

# ---------- Helpers ----------
def parse_html_all_columns(html):
    """Devuelve (filas:list[dict], first_key:str|None)."""
    soup = BeautifulSoup(html, "lxml")
    table = soup.find("table", id="result_list")
    if not table:
        return [], None

    # columnas desde THEAD (clases 'column-<campo>')
    header_keys, thead = [], table.find("thead")
    if thead:
        for th in thead.find_all("th"):
            classes = th.get("class", []) or []
            col = next((c.split("column-")[1] for c in classes
                        if isinstance(c, str) and c.startswith("column-")), None)
            if col:
                header_keys.append(col)
    first_key = header_keys[0] if header_keys else None

    tbody = table.find("tbody")
    if not tbody:
        return [], first_key

    out = []
    for tr in tbody.find_all("tr"):
        row = {}
        # primera celda/th con link al objeto
        th = tr.find("th")
        if th:
            a = th.find("a")
            obj_text = th.get_text(strip=True)
            obj_href = a.get("href") if a else None
            if obj_href:
                obj_href = urljoin(BASE + "/", obj_href)
            row[first_key or "obj"] = obj_text
            row["link"] = obj_href

        # demás celdas <td class="field-xxx">
        for td in tr.find_all("td"):
            classes = td.get("class", []) or []
            fld = next((c.split("field-")[1] for c in classes
                        if isinstance(c, str) and c.startswith("field-")), None)
            if fld:
                row[fld] = td.get_text(strip=True)

        out.append(row)

    return out, first_key

def to_excel_with_format(df_activos: pd.DataFrame, df_dict: pd.DataFrame, xlsx_path: str):
    """Excel con autofiltro, freeze panes y ajuste de anchos."""
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
        # Hoja activos
        df_activos.to_excel(writer, sheet_name="activos", index=False)
        ws = writer.sheets["activos"]
        ws.freeze_panes = "A2"
        ws.auto_filter.ref = ws.dimensions
        for col in ws.columns:
            max_len = 0
            col_letter = col[0].column_letter
            for cell in col:
                try:
                    max_len = max(max_len, len(str(cell.value)) if cell.value is not None else 0)
                except Exception:
                    pass
            ws.column_dimensions[col_letter].width = min(max(10, max_len + 2), 60)

        # Hoja diccionario
        df_dict.to_excel(writer, sheet_name="diccionario", index=False)
        ws2 = writer.sheets["diccionario"]
        ws2.freeze_panes = "A2"
        ws2.auto_filter.ref = ws2.dimensions
        for col in ws2.columns:
            max_len = 0
            col_letter = col[0].column_letter
            for cell in col:
                try:
                    max_len = max(max_len, len(str(cell.value)) if cell.value is not None else 0)
                except Exception:
                    pass
            ws2.column_dimensions[col_letter].width = min(max(10, max_len + 2), 60)

# ----------------- Main -----------------
try:
    all_rows = []
    seen_first_signatures = set()
    global_first_key = None

    for page in range(TOTAL_PAGES):  # 0..53
        url = LIST_TEMPLATE.format(page=page)
        print("Cargando:", url)
        driver.get(url)
        time.sleep(0.7)

        rows, first_key = parse_html_all_columns(driver.page_source)
        if global_first_key is None:
            global_first_key = first_key  # ej. "nombre"

        if not rows:
            print(f"Sin filas en página {page} — corto por seguridad.")
            break

        # Evitar loop si el server repite la última página
        first = rows[0]
        signature = first.get("link") or (first.get("id_bolsa"), first.get(global_first_key or "obj"))
        if signature in seen_first_signatures:
            print("Detectada repetición de página — fin.")
            break
        seen_first_signatures.add(signature)

        all_rows.extend(rows)
        time.sleep(0.2)

    # ---- Guardados JSON/CSV ----
    with open(OUTFILE_JSON, "w", encoding="utf-8") as f:
        json.dump(all_rows, f, ensure_ascii=False, indent=2)

    if all_rows:
        all_keys = sorted({k for r in all_rows for k in r.keys()})
        with open(OUTFILE_CSV, "w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=all_keys)
            w.writeheader()
            w.writerows(all_rows)

    # ---- Diccionario + Excel ----
    display_key = global_first_key or "nombre"
    df_activos = pd.DataFrame(all_rows)

    if "id_bolsa" in df_activos.columns:
        name_col = "nombre" if "nombre" in df_activos.columns else (display_key if display_key in df_activos.columns else "obj")
        df_dict = (
            df_activos.loc[df_activos["id_bolsa"].notna(), ["id_bolsa", name_col]]
            .rename(columns={name_col: "nombre"})
            .drop_duplicates(subset=["id_bolsa"])
            .sort_values("id_bolsa")
        )
        activos_dict = dict(zip(df_dict["id_bolsa"], df_dict["nombre"]))
    else:
        df_dict = pd.DataFrame(columns=["id_bolsa", "nombre"])
        activos_dict = {}

    with open(OUTFILE_DICT, "w", encoding="utf-8") as f:
        json.dump(activos_dict, f, ensure_ascii=False, indent=2)

    to_excel_with_format(df_activos, df_dict, OUTFILE_XLSX)

    print(f"OK: {len(all_rows)} filas → {OUTFILE_JSON}, {OUTFILE_CSV}, {OUTFILE_DICT}, {OUTFILE_XLSX}")

finally:
    try:
        driver.quit()
    except Exception:
        pass


Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=0
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=1
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=2
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=3
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=4
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=5
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=6
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=7
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=8
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=9
Cargando: https://sistema.buenavistacapital.com/admin/informacion_financiera/activo/?p=10
Cargando: https://si